<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/LeNeT_implimentation_from_scratch_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on Google CoLaB with Pytorch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.981)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>>time elapsed: {time_fmt()}")

Mounted at /content/drive/
You are on Google CoLaB with Pytorch version: 1.8.1+cu101
>>>>time elapsed: 0: 02: 03.00


In [2]:
#In this notebook we are going to impliment the LeNet architecture by Lecun and train it on cifar10 data

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import time, sys

In [22]:
#Hyperparameters for the model
batch_size = 64
EPOCHS = 100
learning_rate = 0.001
num_classes = 10


In [13]:
#Lenet Class:
class LeNet(nn.Module):
  def __init__(self,num_classes):
    super(LeNet, self).__init__()
    self.pool = nn.AvgPool2d(kernel_size = (2,2), stride = (2,2))
    self.relu = nn.ReLU()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = (5,5), stride = (1,1), padding = (0,0))
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels =16, kernel_size = (5,5), stride = (1,1), padding = (0,0))
    self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = (5,5), stride = (1,1), padding = (0,0))
    self.fc1 = nn.Linear(in_features = 120, out_features = 84)
    self.fc2 = nn.Linear(in_features = 84, out_features = 10)

  def forward(self, input_tensor):
    x = self.conv1(input_tensor)
    x = self.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.conv3(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x


In [14]:
model = LeNet(num_classes = num_classes).to(device = device)

In [15]:
#Testing to see if the output is of desired shape:
rnd_data = torch.rand(64, 3,32,32).to(device = device)
print(f"The output shape: {model(rnd_data).shape}")

The output shape: torch.Size([64, 10])


In [16]:
#Load the data from torch:
train_data = datasets.CIFAR10(root = '/train_data',train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.CIFAR10(root = "/test_data", train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(dataset = test_data, shuffle = True, batch_size = batch_size)
x_batch, y_batch = next(iter(train_loader))
print(f"x_batch_shape: {x_batch.shape}\ty_batch_shape: {y_batch.shape}")

Files already downloaded and verified
Files already downloaded and verified
x_batch_shape: torch.Size([64, 3, 32, 32])	y_batch_shape: torch.Size([64])


In [23]:
#Get the optimizers and the loss object:
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)
loss_obj = nn.CrossEntropyLoss()

In [24]:
#The training loop:
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>>training start for epoch {epoch + 1}\n>>>>please wait while training...")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    #save data to device
    data = data.to(device = device)
    target = target.to(device = device)
    #forward pass here
    preds = model(data)
    train_loss = loss_obj(preds, target)
    #backward pass here:
    optimizer.zero_grad()
    train_loss.backward()
    #gradient descent with RMSprop optimizer here:
    optimizer.step()

#Model evaluation on the training and validation data
def model_val(loader, model):
  if loader.dataset.train:
    print(f"\n>>>>Checking over the training data\n>>>please wait....")
  else:
    print(f"\n>>>>Checking over the validation data\n>>>>please wait....")
  num_examples = 0
  num_correct = 0
  model.eval()
  #No need to re-compute the gradients
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      preds = model(x)
      _,predictions = preds.max(1)#fetch the maximum proba values in a class of 10
      num_correct+=(predictions == y).sum()#get the total number of correctly classified 
      num_examples+=predictions.size(0)#fetch total number of exaples in a batch(batch_size = 64)
  model.train()
  return float(num_correct/num_examples)*100#fetch the accuracy of the model
toc = time.time()
print(f"\n>>>>The accuracy of this model over the training data is: {model_val(train_loader, model):.2f}")
print(f"\n>>>>The accuracy of this model over the validation data is: {model_val(test_loader, model):.2f}")
print(f"\n>>>>Total time elapsed for training and validation: {time_fmt(toc - tic)}")

  2%|▏         | 12/782 [00:00<00:06, 117.27it/s]


>>>>training start for epoch 1
>>>>please wait while training...


  2%|▏         | 12/782 [00:00<00:06, 118.14it/s]


>>>>training start for epoch 2
>>>>please wait while training...


  2%|▏         | 13/782 [00:00<00:06, 125.95it/s]

...


>>>>training start for epoch 99
>>>>please wait while training...


  2%|▏         | 12/782 [00:00<00:06, 114.73it/s]


>>>>training start for epoch 100
>>>>please wait while training...


100%|██████████| 782/782 [00:06<00:00, 116.51it/s]



>>>>Checking over the training data
>>>please wait....

>>>>The accuracy of this model over the training data is: 88.39

>>>>Checking over the validation data
>>>>please wait....

>>>>The accuracy of this model over the validation data is: 59.16

>>>>Total time elapsed for training and validation: 0: 10: 53.00
